# Modules

In [1]:
import os
import numpy as np

from PIL import Image

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from skimage.feature import greycomatrix, greycoprops

import cv2
import mahotas
import pickle
import cv2
import time


# Utility Functions

In [28]:
def get_image(name, folder):
    filepath = os.path.join(folder, name)
    img = Image.open(filepath)
    return np.array(img)


def get_images(directoryName):
    directory = os.fsencode(directoryName)
    images = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg"):
            image = get_image(filename, directoryName)
            images.append(image)
            continue
        else:
            continue
    return images

def contrast_streching(original_image):
    new_image = original_image.copy()
    min = np.amin(new_image)
    max = np.amax(new_image)
    func = lambda x: (x - min) * (255/(max - min))
    vectorize = np.vectorize(func)
    return vectorize(new_image)

def histogram_equalization(original_image):
    new_image = original_image
    # flatten image and calculate histogram
    histogram_array = np.bincount(original_image.ravel(), minlength=256)

    # normalize
    pixel_count = new_image.shape[0] * new_image.shape[1]
    histogram_array = histogram_array/pixel_count

    # normalized cumulative histogramj
    chistogram_array = np.cumsum(histogram_array)
    
    # pixel mapping
    transform_map = np.floor(255 * chistogram_array).astype(np.uint8)

    # pixel tranformation
    equalized_img_list = [transform_map[i] for i in list(new_image.ravel())]

    # reshaping image
    new_image = np.reshape(np.asarray(equalized_img_list), original_image.shape)
    return new_image

def fd_hu_moments(image):
    # Compute the Hu Moments of the image as a feature
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature


def fd_haralick(image):
    # Compute the haralick texture feature vector from the image
    haralick = mahotas.features.haralick(image).mean(axis=0)
    return haralick


def calc_glcm_all_agls(img, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):

    glcm = greycomatrix(img,
                        distances=dists,
                        angles=agls,
                        levels=lvl,
                        symmetric=sym,
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[
        0]]
    for item in glcm_props:
        feature.append(item)

    return feature

def fd_glcm(image):
    properties = ['dissimilarity', 'correlation',
                  'homogeneity', 'contrast', 'ASM', 'energy']

    glcm_result = calc_glcm_all_agls(image,
                                     props=properties)

    # columns = []
    # angles = ['0', '45', '90', '135']
    # for name in properties:
    #     for ang in angles:
    #         columns.append(name + "_" + ang)

    # columns.append("label")

    return glcm_result

def convolute(input, kernel, pad_width=1, average=False):
    # initial values
    output = np.zeros(input.shape)
    # adding padding
    padding = np.pad(input, pad_width=pad_width, mode='constant', constant_values=0)
    for i in range(input.shape[0]):
        for j in range(input.shape[1]):
            for k in range(kernel.shape[0]):
                for l in range(kernel.shape[1]):
                    output[i][j] += kernel[k][l]*padding[i+k][j+l]
                    if average:
                        output[i][j] /= kernel.shape[0] * kernel.shape[1]
    return output

def sharpen(image):
    blur_kernel = np.array([
        [0.11, 0.11, 0.11],
        [0.11, 0.11, 0.11],
        [0.11, 0.11, 0.11],
    ])

    output = image.copy()

    blurred = convolute(image, blur_kernel)
    alpha = 5
    output = image + alpha * (image - blurred)
    return output

def fd_histogram(image):
    # Compute the histogram of the image as a feature
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    cv2.normalize(hist, hist)
    hist = hist.flatten()
    return hist


def getFeatures(image):
    return np.hstack([fd_histogram(image), fd_glcm(image), fd_hu_moments(image)])
    # return np.hstack([fd_histogram(image), fd_haralick(image), fd_hu_moments(image)])


# Data Loading & Preprocessing

In [3]:
# Get features for the training images with masks
print("Processing training mask images")
mask_images = get_images("images/mask_train")

mask_features = []
for image in mask_images:
    mask_features.append(getFeatures(image))

scaler = MinMaxScaler(feature_range=(0, 1))
# Normalize The feature vectors...
scaler.fit(mask_features)
mask_features = scaler.transform(mask_features)

# Save features to a file
with open('mask_features.pkl', 'wb') as f:
    pickle.dump(mask_features, f)

print("Training Mask Features has shape:", np.shape(mask_features))

# Get features for the training images without masks
print("Processing training no mask images")
nomask_images = get_images("images/unmask_train")

nomask_features = []
for image in nomask_images:
    nomask_features.append(getFeatures(image))

# scaler = MinMaxScaler(feature_range=(0, 1))
# Normalize The feature vectors...
nomask_features = scaler.transform(nomask_features)

# Save features to a file
with open('nomask_features.pkl', 'wb') as f:
    pickle.dump(nomask_features, f)

print("Training No Mask Features has shape:", np.shape(nomask_features))

# Get features for test images with mask
print("Processing test mask images")
mask_images_test = get_images("images/mask_test")

unnormalized_mask_features_test = []
for image in mask_images_test:
    unnormalized_mask_features_test.append(getFeatures(image))

# scaler = MinMaxScaler(feature_range=(0, 1))
# Normalize The feature vectors...
mask_features_test = scaler.transform(unnormalized_mask_features_test)

# Save features to a file
with open('mask_features_test.pkl', 'wb') as f:
    pickle.dump(mask_features_test, f)
print("Testing Mask Features has shape:", np.shape(mask_features_test))

# Get features for test images without mask
print("Processing test no mask images")
nomask_images_test = get_images("images/unmask_test")

unnormalized_nomask_features_test = []
for image in nomask_images_test:
    unnormalized_nomask_features_test.append(getFeatures(image))

# scaler = MinMaxScaler(feature_range=(0, 1))
# Normalize The feature vectors...
print(unnormalized_nomask_features_test)
nomask_features_test = scaler.transform(unnormalized_nomask_features_test)

with open('nomask_features_test.pkl', 'wb') as f:
    pickle.dump(nomask_features_test, f)

print("Testing No Mask Features has shape:",
        np.shape(nomask_features_test))

# Read feature data from file
# with open('mask_features.pkl', 'rb') as f:
#     mask_features = pickle.load(f)
#
# with open('nomask_features.pkl', 'rb') as f:
#     nomask_features = pickle.load(f)
#
# with open('mask_features_test.pkl', 'rb') as f:
#     mask_features_test = pickle.load(f)
#
# with open('nomask_features_test.pkl', 'rb') as f:
#     nomask_features_test = pickle.load(f)


Processing training mask images
Training Mask Features has shape: (617, 287)
Processing training no mask images
Training No Mask Features has shape: (611, 287)
Processing test mask images
Testing Mask Features has shape: (68, 287)
Processing test no mask images
[array([ 1.03211764e-03,  1.03211764e-03,  2.06423528e-03,  2.06423528e-03,
        1.03211764e-03,  0.00000000e+00,  1.03211764e-03,  4.12847055e-03,
        3.09635280e-03,  4.12847055e-03,  2.06423528e-03,  6.19270559e-03,
        4.12847055e-03,  8.25694110e-03,  2.06423528e-03,  5.16058831e-03,
        3.09635280e-03,  7.22482335e-03,  5.16058831e-03,  5.16058831e-03,
        6.19270559e-03,  9.28905886e-03,  1.44496467e-02,  1.44496467e-02,
        1.75460000e-02,  1.85781177e-02,  2.27065887e-02,  1.54817645e-02,
        2.37387065e-02,  2.99314111e-02,  3.30277644e-02,  2.37387065e-02,
        2.06423532e-02,  2.06423532e-02,  1.65138822e-02,  2.06423532e-02,
        1.54817645e-02,  1.54817645e-02,  2.16744710e-02,  2.5

# Model Training

In [4]:
# Read feature data from file
# with open('mask_features.pkl', 'rb') as f:
#     mask_features = pickle.load(f)

# with open('nomask_features.pkl', 'rb') as f:
#     nomask_features = pickle.load(f)

# with open('mask_features_test.pkl', 'rb') as f:
#     mask_features_test = pickle.load(f)

# with open('nomask_features_test.pkl', 'rb') as f:
#     nomask_features_test = pickle.load(f)

# Image labels. Mask = 1.0, No Mask = 0.0
mask_labels = [1.0 for i in mask_features]
nomask_labels = [0.0 for i in nomask_features]
mask_labels_test = [1.0 for i in mask_features_test]
nomask_labels_test = [0.0 for i in mask_features_test]

# Combine mask/no mask features for train and test set
X_train = np.concatenate([mask_features, nomask_features])
y_train = np.concatenate([mask_labels, nomask_labels])

X_test = np.concatenate([mask_features_test, nomask_features_test])
y_test = np.concatenate([mask_labels_test, nomask_labels_test])

# Train the SVM Model
print("Training...")
svm = SVC(kernel='linear', probability=True, random_state=42)
svm.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm.predict(X_test)
print(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Model accuracy is: ', accuracy)


Training...
[[0.         0.         0.         ... 0.58252679 0.18058636 0.65463451]
 [0.00103406 0.         0.0054243  ... 0.58251522 0.18037167 0.65463817]
 [0.         0.         0.00357916 ... 0.58265217 0.18253937 0.65479531]
 ...
 [0.30281242 0.08016414 0.10996901 ... 0.58373423 0.19766948 0.68653796]
 [0.         0.         0.         ... 0.58304362 0.18616041 0.65453779]
 [0.00332304 0.00388109 0.01273834 ... 0.58249708 0.17984461 0.65461217]]
Model accuracy is:  0.8455882352941176


# Testing The Model

In [29]:
size = 4
haar_file = 'haarcascade_frontalface_default.xml'
datasets = 'datasets'

# Part 1: Create fisherRecognizer
print('Recognizing Face Please Be in sufficient Lights...')

# Create a list of images and a list of corresponding names
# (images, labels, names, id) = ([], [], {}, 0)
# for (subdirs, dirs, files) in os.walk(datasets):
#     for subdir in dirs:
#         names[id] = subdir
#         subjectpath = os.path.join(datasets, subdir)
#         for filename in os.listdir(subjectpath):
#             path = subjectpath + '/' + filename
#             label = id
#             images.append(cv2.imread(path, 0))
#             labels.append(int(label))
#         id += 1
(width, height) = (130, 100)

# Create a Numpy array from the two lists above
# (images, labels) = [numpy.array(lis) for lis in [images, labels]]

# OpenCV trains a model from the images
# NOTE FOR OpenCV2: remove '.face'
# model = cv2.face.LBPHFaceRecognizer_create()
# model.train(images, labels)

# Part 2: Use fisherRecognizer on camera stream
# face_cascade = cv2.CascadeClassifier(haar_file)
webcam = cv2.VideoCapture(0)
frame_rate = 10
prev = 0
while True:
    time_elapsed = time.time() - prev
    (_, im) = webcam.read()
    if time_elapsed > 1./frame_rate:
        prev = time.time()

        input_image = cv2.imread("images/custom/ihsan.png")
        gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
        gray = contrast_streching(gray)
        gray = sharpen(gray)
        input_feature = np.array(getFeatures(
            gray))
        input_features = np.array(input_feature)
        # scaler.fit(unnormalized_nomask_features_test)
        input_features = input_features.reshape(1, -1)
        input_features = scaler.transform(input_features)
        print(input_features)
        y_pred = svm.predict(input_features)
        cv2.imshow('OpenCV', input_image)
        print(y_pred)
        # faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        # for (x, y, w, h) in faces:
        #     cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
        #     face = gray[y:y + h, x:x + w]
        #     face_resize = cv2.resize(face, (width, height))

        #     input_feature = np.array(getFeatures(
        #         face_resize))
        #     input_features = np.array(input_feature)
        #     scaler = MinMaxScaler(feature_range=(0, 1))
        #     input_features = input_features.reshape(len(input_feature), 1)
        #     input_features = scaler.fit_transform(
        #         input_features)
        #     input_features = input_features.reshape(1, len(input_feature))
        #     print(input_features)
        #     y_pred = svm.predict(input_features)
        #     # Try to recognize the face
        #     # prediction = svm.predict(face_resize)
        #     cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)

        #     cv2.imshow('OpenCV', im)

        key = cv2.waitKey(10)
        if key == 27:
            break
# input_image = cv2.imread("images/custom/mask_1.png")
# gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
# input_feature = np.array(getFeatures(
#     gray))
# input_features = np.array(input_feature)
# # scaler.fit(unnormalized_nomask_features_test)
# input_features = input_features.reshape(1, -1)
# input_features = scaler.transform(input_features)
# print(input_features)
# y_pred = svm.predict(input_features)
# cv2.imshow('OpenCV', input_image)
# print(y_pred)


Recognizing Face Please Be in sufficient Lights...


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\histogram.cpp:1007: error: (-210:Unsupported format or combination of formats)  in function 'cv::calcHist'
